In [4]:
import numpy as np
import cv2
import time



import numpy as np
import soundfile as sf
import sounddevice as sd
import ipywidgets as wg
from IPython.display import display



#using MIDI note 69 (A440) as a test input
note_test = 69

def MIDI_to_freq(note):
    exp = (note - 69)/ 12
    freq = 2 ** exp * 440
    return freq

def play_audio(data, sr):
    sd.play(data, sr)
    status = sd.wait()
    
def sinusoid(freq, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data


srate = 44100
freq = MIDI_to_freq(69)
data = sinusoid(freq, srate = 44100)


import sys
import ipywidgets as wg
from IPython.display import display

#Sine wav ocsillator that changes frequency in "real-time" based on user text input
#Just type the frequency in Hz you would like to hear then press enter. Enter 0 to quit

amp = 1 
f = 440 #frequency
start_idx = 0
block = 100 #the smaller the value, less latency (but if too low input underflow)
lat = 0 #latency in seconds
srate = 2048


def callback(indata, outdata, frames, time, status):
        if status:
            print(status, file=sys.stderr)
        global start_idx
        t = (start_idx + np.arange(frames)) / srate
        t = t.reshape(-1, 1)
        
        global f
        global amp
        indata = amp * np.sin(2 * np.pi * f * t) #create sine wave
        outdata[:] = indata  
        start_idx += frames



# define a video capture object 
vid = cv2.VideoCapture(0) 
time.sleep(3)

#high speed, low accuraccy 
#tracker = cv2.TrackerMOSSE_create()

#bit slower, more accurate
tracker = cv2.TrackerCSRT_create()

flag = 1

#starts loop
ret, frame = vid.read() 

bbox = cv2.selectROI("Tracking",frame, False)
tracker.init(frame,bbox)

def drawBox(frame,bbox):
    global flag 
    global x_prev, y_prev
    
    x, y, w, h = int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3]) 
    cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (255,0,255), 3,1)
    cv2.putText(frame,"Tracking",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0),2)

    #first frame, no movement
    if flag == 1:
        x_mov = 0
        y_mov = 0
        flag = 0
    else:
        x_mov = x-x_prev
        y_mov = y-y_prev
    
    x_prev = x
    y_prev = y
    return x_mov, y_mov

#original opencv loop is simply put inside of this with satement
with sd.Stream(
                   samplerate=srate, blocksize=block,latency=lat,
                   channels=1, callback=callback ):
    while(True): 
        timer = cv2.getTickCount() 
        # Capture the video frame 
        ret, frame = vid.read() 

        #searches for object in frame
        ret, bbox = tracker.update(frame)

        #if object is found in frame
        if ret:
            x_mov, y_mov = drawBox(frame,bbox)
            #difference in x and y movement in consecutive frames
            
            #method to scale values would be here
            
            #update values
            if(amp+y_mov/100>0 and amp+y_mov/100<1):
                amp = amp +(y_mov/100)
            f = f + x_mov
            print("x change: "+ str(x_mov))
            print("y change: "+ str(y_mov))
        #if object is not found in frame        
        else:
            cv2.putText(frame,"Lost",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)

        fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer)
        cv2.putText(frame,str(int(fps)),(75,50), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)

        # Display the resulting frame 
        cv2.imshow('frame', frame) 

        # press q to quit
        if cv2.waitKey(1) & 0xFF == ord('q'): 
            break

        
        
        
        

        
        
        

# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 


x change: 0
y change: 0
x change: -1
y change: 0
x change: 1
y change: 1
x change: -1
y change: 1
x change: 0
y change: 0
x change: -1
y change: -1
x change: -1
y change: -1
x change: -11
y change: -9
x change: -19
y change: -15
x change: -23
y change: -7
x change: -19
y change: -15
x change: -20
y change: -26
x change: 4
y change: -9
x change: 3
y change: 6
x change: 5
y change: -6
x change: 0
y change: 15
x change: 7
y change: 3
x change: 2
y change: -4
x change: 2
y change: -1
x change: -6
y change: 1
x change: -5
y change: 4
x change: -16
y change: 3
x change: -16
y change: 26
x change: -9
y change: 23
x change: -12
y change: 27
x change: -17
y change: 64
x change: -5
y change: 26
x change: 11
y change: -48
x change: -12
y change: 12
x change: -4
y change: 5
x change: 27
y change: -29
x change: 4
y change: 6
x change: -14
y change: 2
x change: 4
y change: 4
x change: -3
y change: 20
x change: 7
y change: -26
x change: 8
y change: -25
x change: 13
y change: -14
x change: 2
y change: